In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = pd.read_csv('/content/drive/My Drive/ML Project/Nikki/TIDY_unclean.csv')

<ipython-input-2-c27740e93a10>:1: DtypeWarning: Columns (35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/My Drive/ML Project/Nikki/TIDY_unclean.csv')


In [3]:
print(df.columns.tolist())

['projectid', 'teacher_acctid_x', 'schoolid', 'school_ncesid', 'school_latitude', 'school_longitude', 'school_city', 'school_state', 'school_zip', 'school_metro', 'school_district', 'school_county', 'school_charter', 'school_magnet', 'school_year_round', 'school_nlns', 'school_kipp', 'school_charter_ready_promise', 'teacher_prefix', 'teacher_teach_for_america', 'teacher_ny_teaching_fellow', 'primary_focus_subject', 'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'fulfillment_labor_materials', 'total_price_excluding_optional_support', 'total_price_including_optional_support', 'students_reached', 'eligible_double_your_impact_match', 'eligible_almost_home_match', 'date_posted', 'is_exciting', 'at_least_1_teacher_referred_donor', 'fully_funded', 'at_least_1_green_donation', 'great_chat', 'three_or_more_non_teacher_referred_donors', 'one_non_teacher_referred_donor_giving_100_plus', 'donation_from_thoughtful_donor', 'g

In [4]:
df['date_posted'] = pd.to_datetime(df['date_posted'], errors='coerce')

# Filter out any rows posted in 2014 or later
df = df[df['date_posted'] < pd.Timestamp('2014-01-01')]

# Define columns to keep
selected_columns = [
    'school_state',  # renamed as 'state'
    'school_charter',
    'total_price_including_optional_support',
    'total_price_excluding_optional_support',
    'primary_focus_subject',
    'resource_type',
    'poverty_level',
    'grade_level',
    'eligible_double_your_impact_match',
    'eligible_almost_home_match',
    'fully_funded',           # label (will rename to is_fully_funded)
    'projectid',              # keep for reference (not ML input)
    'date_posted'             # keep for date-based split (not ML input)
]

df_filtered = df[selected_columns]

print(f"Filtered data shape: {df_filtered.shape}")
df_filtered.head()

Filtered data shape: (619326, 13)


,school_state,school_charter,total_price_including_optional_support,total_price_excluding_optional_support,primary_focus_subject,resource_type,poverty_level,grade_level,eligible_double_your_impact_match,eligible_almost_home_match,fully_funded,projectid,date_posted
44772,IL,f,522.78,444.36,Special Needs,Other,moderate poverty,Grades 3-5,f,f,t,62526d85d2a1818432d03d600969e99c,2013-12-31
44773,ID,f,274.40,233.24,Mathematics,Supplies,high poverty,Grades 3-5,f,f,f,33d59ac771b80222ad63ef0f4ac47ade,2013-12-31
44774,NH,f,335.40,285.09,Environmental Science,Technology,moderate poverty,Grades 6-8,f,f,f,1a3aaeffc56dd2a421e37d8298024c0a,2013-12-31
44775,VA,f,274.05,232.94,Literacy,Other,highest poverty,Grades PreK-2,f,f,f,33aa19ee4da4c5adf47d0dfb84fab5ef,2013-12-31
44776,IL,f,604.01,513.41,Environmental Science,Supplies,highest poverty,Grades 6-8,t,f,t,e31c0ea8b68f404699dfb0d39e9bc99b,2013-12-31


In [5]:
# Columns to check for outliers
price_cols = [
    'total_price_including_optional_support',
    'total_price_excluding_optional_support'
]

# Step 1: NaN summary
nan_summary = df_filtered.isna().sum().to_frame(name='missing_count')
nan_summary['missing_pct'] = (nan_summary['missing_count'] / len(df_filtered)) * 100

# Step 2: Outlier summary for the two price columns
outlier_summary = pd.DataFrame(columns=['lower_bound', 'upper_bound', 'outlier_count', 'outlier_pct'])

for col in price_cols:
    lower = df_filtered[col].quantile(0.01)
    upper = df_filtered[col].quantile(0.99)
    outlier_mask = (df_filtered[col] < lower) | (df_filtered[col] > upper)
    outlier_count = outlier_mask.sum()
    outlier_pct = outlier_count / len(df_filtered) * 100

    outlier_summary.loc[col] = [lower, upper, outlier_count, outlier_pct]

# Combine both summaries
summary_report = nan_summary.copy()
summary_report[['lower_bound', 'upper_bound', 'outlier_count', 'outlier_pct']] = outlier_summary

# Show full report
summary_report.fillna('', inplace=True)
summary_report


<ipython-input-5-cc7e8361632e>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  summary_report.fillna('', inplace=True)


,missing_count,missing_pct,lower_bound,upper_bound,outlier_count,outlier_pct
school_state,0,0.000000,,,,
school_charter,0,0.000000,,,,
total_price_including_optional_support,0,0.000000,147.56,2822.635,12344.0,1.993134
total_price_excluding_optional_support,0,0.000000,121.0,2366.7175,12325.0,1.990067
primary_focus_subject,36,0.005813,,,,
resource_type,43,0.006943,,,,
poverty_level,0,0.000000,,,,
grade_level,7,0.001130,,,,
eligible_double_your_impact_match,0,0.000000,,,,
eligible_almost_home_match,0,0.000000,,,,


In [6]:
print(df_filtered.shape)
df_cleaned = df_filtered.dropna()
df_cleaned = df_cleaned[(df_cleaned[col] >=  lower) & (df_cleaned[col] <= upper)]
print(df_cleaned.shape)

# Convert t/f → 1/0 for 3 standard columns
binary_cols = [
    'school_charter',
    'eligible_double_your_impact_match',
    'eligible_almost_home_match'
]

for col in binary_cols:
    df_cleaned[col] = df_cleaned[col].map({'t': 1, 'f': 0})

# Convert t/f → 0/1 for the label (fully_funded: 't' means funded → now label 0)
df_cleaned['fully_funded'] = df_cleaned['fully_funded'].map({'t': 0, 'f': 1})

df_cleaned.head()


(619326, 13)
(606954, 13)


,school_state,school_charter,total_price_including_optional_support,total_price_excluding_optional_support,primary_focus_subject,resource_type,poverty_level,grade_level,eligible_double_your_impact_match,eligible_almost_home_match,fully_funded,projectid,date_posted
44772,IL,0,522.78,444.36,Special Needs,Other,moderate poverty,Grades 3-5,0,0,0,62526d85d2a1818432d03d600969e99c,2013-12-31
44773,ID,0,274.40,233.24,Mathematics,Supplies,high poverty,Grades 3-5,0,0,1,33d59ac771b80222ad63ef0f4ac47ade,2013-12-31
44774,NH,0,335.40,285.09,Environmental Science,Technology,moderate poverty,Grades 6-8,0,0,1,1a3aaeffc56dd2a421e37d8298024c0a,2013-12-31
44775,VA,0,274.05,232.94,Literacy,Other,highest poverty,Grades PreK-2,0,0,1,33aa19ee4da4c5adf47d0dfb84fab5ef,2013-12-31
44776,IL,0,604.01,513.41,Environmental Science,Supplies,highest poverty,Grades 6-8,1,0,0,e31c0ea8b68f404699dfb0d39e9bc99b,2013-12-31


In [7]:
df_gdp = pd.read_csv('/content/drive/My Drive/ML Project/Nikki/SASUMMARY__ALL_AREAS_1998_2024.csv')
df_gdp_filtered = df_gdp[df_gdp['Description'] == ' Gross domestic product (GDP) ']

df_gdp_filtered = df_gdp_filtered.drop(['GeoFIPS', 'Region', 'TableName', 'LineCode', 'IndustryClassification'], axis=1)
df_gdp_filtered = df_gdp_filtered.drop(3, axis=0)
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "Virgin Islands, U.S.": "VI",
}
df_gdp_filtered['GeoName'] = df_gdp_filtered['GeoName'].replace(us_state_to_abbrev)

df_gdp_filtered.head()

,GeoName,Description,Unit,1998,1999,2000,2001,2002,2003,2004,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
18,AL,Gross domestic product (GDP),Millions of current dollars,110838.8,116514.3,120522.6,123534.5,128380.9,135212.6,148379.1,...,203113.3,208824.3,216615.5,226263.8,234798.2,235325.3,260018.4,284949.1,304935.6,321237.6
33,AK,Gross domestic product (GDP),Millions of current dollars,24387.8,24920.4,26934,28660.9,29884.5,32115.8,35389.5,...,51574.2,51121.8,53550.9,54762,54549.6,51283.4,58943.5,66271.9,68056,69969
48,AZ,Gross domestic product (GDP),Millions of current dollars,144393.2,156442.1,165735.6,172899.6,180844.5,194318,207246.3,...,298942.2,313787.1,333099,353671,375376.7,388956.7,436778.6,484052.2,522767.2,552167
63,AR,Gross domestic product (GDP),Millions of current dollars,62852,67279,69176.3,71099.6,74453.1,79009.7,85928.6,...,118573.6,120987.4,123882.6,129213.8,132625.8,136198.4,154052.5,168605.8,178606.4,188723.1
78,CA,Gross domestic product (GDP),Millions of current dollars,1143672.5,1239773.5,1354791.3,1373270.8,1419321.8,1498578,1585081.4,...,2487156.4,2586485.2,2740550.3,2899530.9,3068629.7,3076755.1,3423958.3,3660415.7,3870378.9,4103123.6


In [8]:
# Extract the year from the date_posted column
df_cleaned['year_posted'] = df_cleaned['date_posted'].dt.year

# Melt the GDP dataframe so years become rows
df_gdp_melted = df_gdp_filtered.melt(
    id_vars=['GeoName'],
    value_vars=[str(year) for year in range(2002, 2014)],
    var_name='year_posted',
    value_name='state_gdp'
)

# Convert year_posted to numeric for matching
df_gdp_melted['year_posted'] = df_gdp_melted['year_posted'].astype(int)

# Merge GDP values into df_filtered
df_merged = df_cleaned.merge(
    df_gdp_melted,
    how='left',
    left_on=['school_state', 'year_posted'],
    right_on=['GeoName', 'year_posted']
)

# Drop GeoName column after merge if desired
df_merged = df_merged.drop(columns=['GeoName'])
df_merged.head()

,school_state,school_charter,total_price_including_optional_support,total_price_excluding_optional_support,primary_focus_subject,resource_type,poverty_level,grade_level,eligible_double_your_impact_match,eligible_almost_home_match,fully_funded,projectid,date_posted,year_posted,state_gdp
0,IL,0,522.78,444.36,Special Needs,Other,moderate poverty,Grades 3-5,0,0,0,62526d85d2a1818432d03d600969e99c,2013-12-31,2013,742890.9
1,ID,0,274.40,233.24,Mathematics,Supplies,high poverty,Grades 3-5,0,0,1,33d59ac771b80222ad63ef0f4ac47ade,2013-12-31,2013,61712.6
2,NH,0,335.40,285.09,Environmental Science,Technology,moderate poverty,Grades 6-8,0,0,1,1a3aaeffc56dd2a421e37d8298024c0a,2013-12-31,2013,70897.4
3,VA,0,274.05,232.94,Literacy,Other,highest poverty,Grades PreK-2,0,0,1,33aa19ee4da4c5adf47d0dfb84fab5ef,2013-12-31,2013,456911.6
4,IL,0,604.01,513.41,Environmental Science,Supplies,highest poverty,Grades 6-8,1,0,0,e31c0ea8b68f404699dfb0d39e9bc99b,2013-12-31,2013,742890.9


In [10]:
df_merged = df_merged.drop(columns = ['state_gdp'])

In [11]:
df_merged['date_posted'] = pd.to_datetime(df_merged['date_posted'], errors='coerce')
train_mask = df_merged['date_posted'] < pd.Timestamp('2013-01-01')
test_mask = ~train_mask
df_model = df_merged.copy()

target_col = 'fully_funded'

columns_to_drop = ['projectid', 'date_posted', 'fully_funded']
feature_cols = [col for col in df_merged.columns if col not in columns_to_drop]

X_train = df_model.loc[train_mask, feature_cols]
y_train = df_model.loc[train_mask, target_col]

X_test = df_model.loc[test_mask, feature_cols]
y_test = df_model.loc[test_mask, target_col]

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Target distribution in train:\n{y_train.value_counts(normalize=True)}")

Train shape: (477227, 11), Test shape: (129727, 11)
Target distribution in train:
fully_funded
0    0.693353
1    0.306647
Name: proportion, dtype: float64


In [12]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [13]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42)
}

# Train & evaluate each
for name, model in models.items():
    print(f"\nTraining: {name}")

    clf = Pipeline([
        ('preprocess', preprocessor),
        ('model', model)
    ])

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:, 1]

    print(f"{name} AUC: {roc_auc_score(y_test, y_prob):.4f}")
    print(f"{name} Classification Report:\n", classification_report(y_test, y_pred))


Training: Logistic Regression
Logistic Regression AUC: 0.6674
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.88      0.80     91534
           1       0.47      0.25      0.33     38193

    accuracy                           0.70    129727
   macro avg       0.60      0.57      0.57    129727
weighted avg       0.66      0.70      0.66    129727


Training: Random Forest
Random Forest AUC: 0.6342
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.78      0.76     91534
           1       0.41      0.37      0.39     38193

    accuracy                           0.66    129727
   macro avg       0.58      0.58      0.58    129727
weighted avg       0.65      0.66      0.65    129727


Training: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:45:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost AUC: 0.6866
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.85      0.80     91534
           1       0.48      0.32      0.39     38193

    accuracy                           0.70    129727
   macro avg       0.62      0.59      0.59    129727
weighted avg       0.67      0.70      0.68    129727


Training: AdaBoost
AdaBoost AUC: 0.6675
AdaBoost Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.86      0.80     91534
           1       0.45      0.27      0.34     38193

    accuracy                           0.69    129727
   macro avg       0.60      0.57      0.57    129727
weighted avg       0.65      0.69      0.66    129727



In [14]:
# predict_proba
y_proba = clf.predict_proba(X_test)[:, 1]

# set threshold to 0.3
threshold = 0.3
y_pred_custom = (y_proba >= threshold).astype(int)

print(f"Custom threshold @ {threshold}")
print(classification_report(y_test, y_pred_custom))
print(f"AUC: {roc_auc_score(y_test, y_proba):.4f}")


Custom threshold @ 0.3
              precision    recall  f1-score   support

           0       0.93      0.11      0.20     91534
           1       0.32      0.98      0.48     38193

    accuracy                           0.37    129727
   macro avg       0.62      0.55      0.34    129727
weighted avg       0.75      0.37      0.28    129727

AUC: 0.6675


In [15]:
def evaluate_threshold(y_true, y_proba, threshold=0.5):
    print(f"\nEvaluating at threshold = {threshold:.2f}")
    y_pred = (y_proba >= threshold).astype(int)

    print("Classification Report:")
    print(classification_report(y_true, y_pred, digits=4))

    auc = roc_auc_score(y_true, y_proba)
    print(f"AUC: {auc:.4f}")
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

for thresh in [0.5, 0.4, 0.35, 0.3, 0.25]:
    evaluate_threshold(y_test, y_proba, threshold=thresh)



Evaluating at threshold = 0.50
Classification Report:
              precision    recall  f1-score   support

           0     0.7391    0.8644    0.7968     91534
           1     0.4525    0.2686    0.3371     38193

    accuracy                         0.6890    129727
   macro avg     0.5958    0.5665    0.5670    129727
weighted avg     0.6547    0.6890    0.6615    129727

AUC: 0.6675
Confusion Matrix:
[[79118 12416]
 [27933 10260]]

Evaluating at threshold = 0.40
Classification Report:
              precision    recall  f1-score   support

           0     0.8361    0.4644    0.5971     91534
           1     0.3785    0.7818    0.5101     38193

    accuracy                         0.5578    129727
   macro avg     0.6073    0.6231    0.5536    129727
weighted avg     0.7014    0.5578    0.5715    129727

AUC: 0.6675
Confusion Matrix:
[[42507 49027]
 [ 8332 29861]]

Evaluating at threshold = 0.35
Classification Report:
              precision    recall  f1-score   support

    